In [21]:
import os
import time
import random
import numpy as np
import pandas as pd
import librosa

import tensorflow as tf
from tensorflow.python.platform import gfile

In [22]:
"""
DAAATASET
"""
DEBUG = False
PATH_TRAIN = os.getcwd()+'/data/train'
PATH_TEST = os.getcwd()+'/data/test'
BATCH_SIZE = 10
ITERATIONS = 2000
ITERATIONS_TEST = 10
EVAL_EVERY = 5
HEIGHT = 25
WIDTH = 64
PAD_WIDTH = 300
NUM_LABELS = 0
LEARNING_RATE = 1E-4
LOGDIR = 'log/'
TEST_LOGDIR = 'log_test/'
LABEL_TO_INDEX_MAP = {}

In [23]:
def init(path):
    labels = os.listdir(path)
    index = 0
    for label in labels:
        LABEL_TO_INDEX_MAP[label] = index
        index += 1
        
    global NUM_LABELS
    NUM_LABELS = len(LABEL_TO_INDEX_MAP)
    print(LABEL_TO_INDEX_MAP)

In [24]:
def one_hot_encoding(label):
    encoding = [0] * len(LABEL_TO_INDEX_MAP)
    encoding[LABEL_TO_INDEX_MAP[label]] = 1
    return encoding

In [25]:
def get_mfcc(wave_path, PAD_WIDTH=WIDTH):
    if DEBUG:
        print(wave_path)
    wave, sr = librosa.load(wave_path, mono=True, duration=0.7)
    mfccs = librosa.feature.mfcc(y=wave, sr=sr, n_mfcc=HEIGHT)
    mfccs = np.pad(mfccs,((0,0), (0, PAD_WIDTH - len(mfccs[0]))), mode='constant')
    return mfccs

In [26]:
def get_batch(batch_size, path):
    X = []
    Y = []
    random.seed(5896)
    path = os.path.join(path,'*', '*.wav')
    waves = gfile.Glob(path)
    while True:
        random.shuffle(waves)
        for wave_path in waves:
            _,label = os.path.split(os.path.dirname(wave_path))
            X.append(get_mfcc(wave_path))
            Y.append(one_hot_encoding(label))

            if (len(X) == batch_size):
                yield X, Y
                X = []
                Y = []

In [27]:
def get_model(input, dropout):
    #topology
    #.1. Convolutional Layer -> Relu -> dropout -> maxpool
    # 2. Convolutional layer -> relu-> dropout
    # 3.fully connected layer -> matmul
    
    #first convolutional
    with tf.name_scope('Conv1'):
        input_4D = tf.reshape(input, [-1, HEIGHT, WIDTH, 1])
        w1 = tf.Variable(tf.truncated_normal([12,8,1,44], stddev=0.01), name="W")
        b1 = tf.Variable(tf.zeros([44]),name="B")
        conv1 = tf.nn.conv2d(input_4D, w1, strides=[1,1,1,1],padding="SAME" )
        act1 = tf.nn.relu(conv1 + b1)
        drop1 = tf.nn.dropout(act1, dropout)
        max_pool1 = tf.nn.max_pool(drop1,ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
        tf.summary.histogram('weight',w1)
        tf.summary.histogram('biases',b1)
        tf.summary.histogram('activations', act1)
        tf.summary.histogram('dropouts',drop1)
        
    with tf.name_scope('Conv2'):
        w2 = tf.Variable(tf.truncated_normal([6,4,44,44],stddev=0.01), name="W")
        b2 = tf.Variable(tf.zeros([44]), name="B")
        conv2 = tf.nn.conv2d(max_pool1, w2, strides=[1,1,1,1],padding='SAME')
        act2 = tf.nn.relu(conv2+b2)
        drop2 = tf.nn.dropout(act2, dropout)
        tf.summary.histogram('weight',w2)
        tf.summary.histogram("biases",b2)
        tf.summary.histogram('activations',act2)
        tf.summary.histogram('dropouts',drop2)
        
    #Reshaping for Fully connected layer
    conv_shape = drop2.get_shape()
    count = int(conv_shape[1] * conv_shape[2] * conv_shape[3])
    flat_output = tf.reshape(drop2,[-1,count])
    
    #fully connected layer
    with tf.name_scope('FC'):
        w3 = tf.Variable(tf.truncated_normal([count,NUM_LABELS],stddev=0.01))
        b3 = tf.Variable(tf.zeros([NUM_LABELS]))
        fc = tf.add(tf.matmul(flat_output,w3),b3)
        tf.summary.histogram('weight',w3)
        tf.summary.histogram('biases',b3)
        
    return fc
    

In [28]:
def main():
    #starting tensorflow graph and session
    tf.reset_default_graph()
    sess = tf.Session()
    
    #placeholder for input
    x = tf.placeholder(tf.float32, shape=[None, HEIGHT, WIDTH], name='input')
    
    #placeholder for label
    y = tf.placeholder(tf.float32, shape=[None, NUM_LABELS], name='label')
    
    #placeholder for dropout
    dropout = tf.placeholder(tf.float32, name='dropout')
    
    #NN Model
    logits = get_model(x, dropout)
    
    #loss function
    with tf.name_scope('loss'):
        #watchout this line
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y), name='loss')
        tf.summary.scalar('loss', loss)
        
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)
    
    with tf.name_scope('accuracy'):
        predicted = tf.argmax(logits,1)
        truth = tf.argmax(y,1)
        correct_prediction = tf.equal(predicted,truth)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        confusion_matrix = tf.confusion_matrix(truth, predicted, num_classes=NUM_LABELS)
        tf.summary.scalar('accuracy',accuracy)
    
    summ = tf.summary.merge_all()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR)
    writer.add_graph(sess.graph)
    test_writer = tf.summary.FileWriter(TEST_LOGDIR)
    
    #Training model
    print("starting Training\n")
    batch = get_batch(BATCH_SIZE, PATH_TRAIN)
    start_time = time.time()
    for i in range(1, ITERATIONS + 1):
        X, Y = next(batch)
        if i % EVAL_EVERY == 0:
            #watch this dropout
            [train_accuracy, train_loss, s] = sess.run([accuracy, loss, summ], feed_dict={x: X, y: Y, dropout: 0.2})
            acc_and_loss = [i, train_loss, train_accuracy * 100]
            print('Iteration # {}. Train Loss {:,.2f}. Train Acc: {:,.0f}%'.format(*acc_and_loss))
            writer.add_summary(s, i)
        if i % (EVAL_EVERY * 20) == 0:
            train_confusion_matrix = sess.run([confusion_matrix], feed_dict={x: X, y: Y, dropout: 0.3})
            header=LABEL_TO_INDEX_MAP.keys()
            #watch this droput
            df=pd.DataFrame(np.reshape(train_confusion_matrix, (NUM_LABELS, NUM_LABELS)), index=header)
            print('\nConfusion Matrix:\n {}\n'.format(df))
            saver.save(sess, os.path.join(LOGDIR, 'model.ckpt'), i)
            
        sess.run(train_step, feed_dict={x: X, y: Y, dropout: 0.2})
    
    print('\nTotal training time: {:0f} seconds\n'.format(time.time() - start_time))
    
#def test():
    #testing model
#    init(PATH_TEST)
    batch = get_batch(BATCH_SIZE, PATH_TEST)
    total_accuracy = 0
    for i in range(ITERATIONS_TEST):
        X, Y = next(batch, PATH_TEST)
        print("tested.")
        #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        #summ = tf.summary.merge_all()

        test_accuracy, s = sess.run([accuracy, summ], feed_dict={x: X, y: Y, dropout: 0.9})
        print('Iteration # {}. Test Accuracy: {:.0f}%'.format(i+1, test_accuracy * 100))
        total_accuracy += (test_accuracy / ITERATIONS_TEST)
        test_writer.add_summary(s,i)
        
    print('\nFinal Test Accuracy: {:.0f}%'.format(total_accuracy * 100))


In [48]:
if __name__ == '__main__':
    init(PATH_TRAIN)
    main()


{'ba': 0, 'ju': 1, 'mi': 2, 'nu': 3, 'si': 4, 'ta': 5}
starting Training

Iteration # 5. Train Loss 5.29. Train Acc: 0%
Iteration # 10. Train Loss 3.36. Train Acc: 10%
Iteration # 15. Train Loss 3.12. Train Acc: 10%
Iteration # 20. Train Loss 2.44. Train Acc: 40%
Iteration # 25. Train Loss 3.02. Train Acc: 10%
Iteration # 30. Train Loss 3.09. Train Acc: 20%
Iteration # 35. Train Loss 2.01. Train Acc: 20%
Iteration # 40. Train Loss 2.23. Train Acc: 30%
Iteration # 45. Train Loss 1.56. Train Acc: 30%
Iteration # 50. Train Loss 2.77. Train Acc: 10%
Iteration # 55. Train Loss 1.44. Train Acc: 40%
Iteration # 60. Train Loss 1.73. Train Acc: 20%
Iteration # 65. Train Loss 1.51. Train Acc: 20%
Iteration # 70. Train Loss 1.79. Train Acc: 30%
Iteration # 75. Train Loss 1.51. Train Acc: 40%
Iteration # 80. Train Loss 1.92. Train Acc: 30%
Iteration # 85. Train Loss 1.47. Train Acc: 50%
Iteration # 90. Train Loss 1.42. Train Acc: 30%
Iteration # 95. Train Loss 1.57. Train Acc: 30%
Iteration # 100.

Iteration # 715. Train Loss 0.04. Train Acc: 100%
Iteration # 720. Train Loss 0.08. Train Acc: 100%
Iteration # 725. Train Loss 0.02. Train Acc: 100%
Iteration # 730. Train Loss 0.03. Train Acc: 100%
Iteration # 735. Train Loss 0.01. Train Acc: 100%
Iteration # 740. Train Loss 0.02. Train Acc: 100%
Iteration # 745. Train Loss 0.15. Train Acc: 90%
Iteration # 750. Train Loss 0.04. Train Acc: 100%
Iteration # 755. Train Loss 0.00. Train Acc: 100%
Iteration # 760. Train Loss 0.04. Train Acc: 100%
Iteration # 765. Train Loss 0.19. Train Acc: 90%
Iteration # 770. Train Loss 0.07. Train Acc: 100%
Iteration # 775. Train Loss 0.26. Train Acc: 90%
Iteration # 780. Train Loss 0.17. Train Acc: 90%
Iteration # 785. Train Loss 0.07. Train Acc: 100%
Iteration # 790. Train Loss 0.04. Train Acc: 100%
Iteration # 795. Train Loss 0.34. Train Acc: 90%
Iteration # 800. Train Loss 0.03. Train Acc: 100%

Confusion Matrix:
     0  1  2  3  4  5
ba  2  0  0  0  0  0
ju  0  3  0  0  0  0
mi  0  0  1  0  0  0
n

Iteration # 1415. Train Loss 0.00. Train Acc: 100%
Iteration # 1420. Train Loss 0.02. Train Acc: 100%
Iteration # 1425. Train Loss 0.00. Train Acc: 100%
Iteration # 1430. Train Loss 0.00. Train Acc: 100%
Iteration # 1435. Train Loss 0.01. Train Acc: 100%
Iteration # 1440. Train Loss 0.02. Train Acc: 100%
Iteration # 1445. Train Loss 0.01. Train Acc: 100%
Iteration # 1450. Train Loss 0.00. Train Acc: 100%
Iteration # 1455. Train Loss 0.00. Train Acc: 100%
Iteration # 1460. Train Loss 0.00. Train Acc: 100%
Iteration # 1465. Train Loss 0.02. Train Acc: 100%
Iteration # 1470. Train Loss 0.01. Train Acc: 100%
Iteration # 1475. Train Loss 0.00. Train Acc: 100%
Iteration # 1480. Train Loss 0.01. Train Acc: 100%
Iteration # 1485. Train Loss 0.00. Train Acc: 100%
Iteration # 1490. Train Loss 0.14. Train Acc: 90%
Iteration # 1495. Train Loss 0.02. Train Acc: 100%
Iteration # 1500. Train Loss 0.00. Train Acc: 100%

Confusion Matrix:
     0  1  2  3  4  5
ba  2  0  0  0  0  0
ju  0  2  0  0  0  0


In [49]:
#try restoring model.

mysaver = tf.train.Saver()
with tf.Session() as sess:
    mysaver.restore(sess, os.path.join(LOGDIR,'endmodel.ckpt'))
    

INFO:tensorflow:Restoring parameters from log/endmodel.ckpt


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for log/endmodel.ckpt
	 [[Node: save_1/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save_1/Const_0_0, save_1/RestoreV2/tensor_names, save_1/RestoreV2/shape_and_slices)]]

Caused by op 'save_1/RestoreV2', defined at:
  File "C:\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykern